In [72]:
import pandas as pd
import numpy as np
import os
import json

In [73]:
dir_name = 'C:\\Users\\nstan\Downloads\\'
filename_json = f'{dir_name}gtfsr_connect.json'

In [74]:
with open(filename_json) as json_file:
    df_json = json.load(json_file)

In [75]:
# not necessary
Entity = pd.DataFrame(
    data=[(e['Id'], e['IsDeleted']) for e in df_json['Entity'] if e['Id'][1] != '#'],
    columns=['Id', 'IsDeleted']
)
Entity

,Id,IsDeleted
0,254163638,False
1,282351984,False
2,282385362,False
3,264505093,False
4,282384857,False
...,...,...
2527,282384687,False
2528,282181087,False
2529,282351593,False
2530,282181685,False


In [76]:
Trip = pd.json_normalize(df_json['Entity'])
Trip = Trip.loc[[not x.startswith('#') for x in Trip['Id']],:]
Trip.head(5)

,Id,IsDeleted,TripUpdate.Trip.TripId,TripUpdate.Trip.RouteId,TripUpdate.Trip.StartTime,TripUpdate.Trip.StartDate,TripUpdate.Trip.ScheduleRelationship,TripUpdate.StopTimeUpdate
0,254163638,False,254163638,43975_3,19:44:00,20230508,Scheduled,"[{'StopId': '000009057862', 'Departure': {'Del..."
1,282351984,False,282351984,43245_2,17:12:00,20230509,Scheduled,"[{'StopId': '000009013478', 'Departure': {'Del..."
2,282385362,False,282385362,43195_2,20:03:00,20230508,Scheduled,"[{'StopId': '000009303000', 'Departure': {'Del..."
3,264505093,False,264505093,59935_3,19:12:00,20230508,Scheduled,"[{'StopId': '000009057500', 'Departure': {'Del..."
4,282384857,False,282384857,43216_2,04:37:00,20230509,Scheduled,"[{'StopId': '000009049320', 'Departure': {'Del..."


In [77]:
Stop_time_update = pd.DataFrame(index=Trip['TripUpdate.Trip.TripId'])

In [78]:
for trip_id, stop_time_updates in zip(Trip['TripUpdate.Trip.TripId'], Trip['TripUpdate.StopTimeUpdate']):
    for stop_time_update in stop_time_updates:
        for field, value in stop_time_update.items():
            if field in ['Arrival', 'Departure']:
                for field2, value2 in value.items():
                    Stop_time_update.loc[trip_id, f'{field}.{field2}'] = value2
            else:
                Stop_time_update.loc[trip_id, field] = value
Stop_time_update = Stop_time_update[['StopId', 'StopSequence', 'Arrival.Delay', 'Arrival.Time',
                                     'Departure.Delay', 'Departure.Time', 'ScheduleRelationship']]
Stop_time_update

,StopId,StopSequence,Arrival.Delay,Arrival.Time,Departure.Delay,Departure.Time,ScheduleRelationship
TripUpdate.Trip.TripId,,,,,,,
254163638,000009057862,NaN,NaN,NaN,0.0,NaN,Scheduled
282351984,000009013478,NaN,NaN,NaN,0.0,NaN,Scheduled
282385362,000009058008,5.0,-60.0,NaN,0.0,NaN,Scheduled
264505093,000009050640,11.0,30.0,NaN,30.0,NaN,Scheduled
282384857,000009049320,NaN,NaN,NaN,0.0,NaN,Scheduled
...,...,...,...,...,...,...,...
282384687,000009044660,NaN,NaN,NaN,0.0,NaN,Scheduled
282181087,000009046671,NaN,NaN,NaN,0.0,NaN,Scheduled
282351593,000008000049,NaN,NaN,NaN,0.0,NaN,Scheduled


In [79]:
Trip.drop('TripUpdate.StopTimeUpdate', axis=1, inplace=True)

In [80]:
with pd.ExcelWriter('GTFS_vbn_json.xlsx') as wr:
    Trip.to_excel(wr, sheet_name='Trip', index=False)
    Stop_time_update.to_excel(wr, sheet_name='StopTimeUpdate', index=True)

In [ ]:
# RouteId => split